In [29]:
# rl-stats-eda notebook
# Import statements
import os
import sys
sys.path.insert(0, os.path.abspath('../src'))
import json
import scraper as s
import numpy as np
import pandas as pd
from pymongo import MongoClient
from pymongo.errors import DuplicateKeyError

# Define MongoDB database and collection
db_client = MongoClient('localhost', 27017)
db = db_client['rocket_league']
rl_replays = db['rl_replays']

In [30]:
s.ping_api()

200

In [31]:
data = {
        'pro': True,
    }

one = s.get_replay_list(data)

In [32]:
one_replay_id = one["list"][0]["id"]
one_replay_id

'de63ecf8-9903-40c1-ab63-0d38d9e5c732'

In [33]:
one_replay_document = s.get_specific_replay(one_replay_id)

In [34]:
one_replay_document.keys()

dict_keys(['id', 'link', 'created', 'uploader', 'status', 'rocket_league_id', 'match_guid', 'title', 'map_code', 'match_type', 'team_size', 'playlist_id', 'duration', 'overtime', 'season', 'date', 'visibility', 'blue', 'orange', 'playlist_name', 'map_name'])

In [35]:
tmp = []

for i,rep_id in enumerate(one["list"]):
    #print(f'{i}  {rep_id}'')
    print(i)
    print(one["list"][i]["id"])
    tmp.append(one["list"][i]["id"])


#one["list"][2]["id"]

0
de63ecf8-9903-40c1-ab63-0d38d9e5c732
1
1ecaa5b0-fc82-4879-9434-74dbc14cf944
2
34b8faae-a2b6-45d8-a48f-0e77f4b9a55e
3
6af2f503-6de6-4a87-a905-f9aa99849e66
4
2b7ee582-2f69-4ecf-af4d-b37a97db9a43
5
26175d9d-c7f8-4477-8308-ba6807040f43
6
895d4506-12fa-48ff-aabc-303d4273389d
7
73b9f9c2-33c8-45f1-927f-a764f0f3ff88
8
fad55edc-0811-4192-9b96-12a86795437a
9
9bae01c4-b936-417c-8ccf-fe8500721e06
10
f25d8a96-8ae6-439e-8b74-11aaee556e6b
11
790cd2da-1b72-4f90-bbd1-870b96a8400c
12
ce368746-1c96-497d-92cf-333620843aa2
13
b8723bbe-0278-47b7-93a2-454e4ea788c5
14
66604290-e4d5-4ce0-bfaf-b7deeef8a027
15
7c8b2515-87cf-4b6d-9b63-0588cd01e427
16
43ed3b45-010e-4fea-89e9-72cd3fc0b92c
17
5ca8c94f-65ce-4f24-b5c4-3ee0bb80b0f1
18
86dde117-6904-41a0-8487-a23a430b0296
19
35af312f-5d0b-4bb7-b649-77c36293d3d1
20
8db4a93f-2c8c-4d1c-8acf-76ad3ab8e746
21
b0efe32e-d813-4a4c-a448-e58c63163865
22
75271e23-52f8-4607-8834-c8e00ec7623e
23
413f86ee-4e69-4671-b67b-8a6b2b5104df
24
ea08f4db-5792-4897-a3ba-f68cef6def6f
25
8178c85

In [36]:
len(tmp)

50

### MongoDB Testing

In [37]:
db.rl_replays.count()

466

In [38]:
test = db.rl_replays.find_one()
test2 = db.rl_replays.find_one()

In [39]:
test.keys()

dict_keys(['_id', 'id', 'link', 'created', 'uploader', 'status', 'rocket_league_id', 'match_guid', 'title', 'map_code', 'match_type', 'team_size', 'playlist_id', 'duration', 'overtime', 'season', 'date', 'visibility', 'blue', 'orange', 'playlist_name', 'map_name'])

In [40]:
"""
Currently doing in memory, but would like to send data to collection
**FOR ONE ENTRY IN NEW COLLECTION**
GAME STATS:
replay_id

PLAYER STATS (initialized with mongo):
name, car_name, goals, demo_inflicted, demo_taken
 

TEAM STATS (created from PLAYER STATS in pandas):
blue_total_goals, blue_total_demos_inflicted, blue_total_demos_taken
orange_total_goals, orange_total_demos_inflicted, orange_total_demos_taken

"""

'\nCurrently doing in memory, but would like to send data to collection\n**FOR ONE ENTRY IN NEW COLLECTION**\nGAME STATS:\nreplay_id\n\nPLAYER STATS (initialized with mongo):\nname, car_name, goals, demo_inflicted, demo_taken\n \n\nTEAM STATS (created from PLAYER STATS in pandas):\nblue_total_goals, blue_total_demos_inflicted, blue_total_demos_taken\norange_total_goals, orange_total_demos_inflicted, orange_total_demos_taken\n\n'

In [41]:
subset = db.rl_replays.count() // 8
# all_replays = db.rl_replays.count()    # working with subset for now

subset

58

In [42]:
# get clean data from every replay



df_cols = ["blue_total_goals",
    "blue_total_demos_inflicted",
    "blue_total_demos_taken",
    "orange_total_goals",
    "orange_total_demos_inflicted",
    "orange_total_demos_taken"]

r_data = np.empty((db.rl_replays.count(), len(df_cols)))

r_id = np.empty((r_data.shape[0]), dtype=str)


print(r_data.shape)
len(r_id)

(466, 6)


466

In [47]:
# get relevant info from database
replay_cursor = db.rl_replays.find({}, 
    {"_id": 0,
    "id":1,
    "blue":1,
    "orange":1})

len(replay_cursor[0]["blue"]["players"])

3

In [17]:
for row,replay in enumerate(replay_cursor):
    #r_id[0].join(replay["id"])
    
    # Get info for Blue Team
    for player in replay["blue"]["players"]:
        r_data[row][0] += player["stats"]["core"]["goals"]
        r_data[row][1] += player["stats"]["demo"]["inflicted"]
        r_data[row][2] += player["stats"]["demo"]["taken"]
    # Get info for Orange Team
    for player in replay["orange"]["players"]:
        r_data[row][3] += player["stats"]["core"]["goals"]
        r_data[row][4] += player["stats"]["demo"]["inflicted"]
        r_data[row][5] += player["stats"]["demo"]["taken"]


KeyError: 'blue'

In [81]:
"""
OLD TEST
t_curs[0]["blue"]["players"]    # len 3

for player in t_curs[0]["blue"]["players"]:
    print(f'{player["name"]}\
        {player["car_name"]}')
for player in t_curs[0]["orange"]["players"]:
    print(f'{player["name"]}\
        {player["car_name"]}')
"""

Labdoman        Octane
KAI        Octane
Squishy Nugget        Fennec
Woody        Octane
Dodongo        Octane
kaducious        Octane


In [153]:
#print(replay_cursor[0]["blue"]["players"][0]["stats"]["core"]["goals"])
#print(replay_cursor[0]["blue"]["players"][0]["stats"]["demo"])
#print(replay_cursor[0])

InvalidOperation: cannot set options after executing query